# Convert Models to TensorFlow Lite
## For Raspberry Pi edge deployment

## Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from pathlib import Path

print(f'TensorFlow version: {tf.__version__}')

## Convert MobileNetV2 (Edge Device)

In [ ]:
# Load MobileNetV2
model_mobile = tf.keras.models.load_model('models/farmog_disease_classifier.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_mobile)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save
with open('models/mobilenet_v2.tflite', 'wb') as f:
    f.write(tflite_model)

mobile_size = len(tflite_model) / (1024*1024)
print(f'✅ MobileNetV2 converted: {mobile_size:.2f} MB')

## Convert ResNet50V2 (Best Accuracy)

In [ ]:
# Load ResNet50V2
model_resnet = tf.keras.models.load_model('models/farmog_resnet50v2_classifier.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model_resnet)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_resnet = converter.convert()

# Save
with open('models/resnet50v2.tflite', 'wb') as f:
    f.write(tflite_resnet)

resnet_size = len(tflite_resnet) / (1024*1024)
print(f'✅ ResNet50V2 converted: {resnet_size:.2f} MB')

## Test TFLite Model

In [ ]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path='models/mobilenet_v2.tflite')
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print('Input shape:', input_details[0]['shape'])
print('Output shape:', output_details[0]['shape'])

# Test inference
test_input = np.random.rand(1, 224, 224, 3).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

print('\n✅ TFLite model working!')
print(f'Output shape: {output.shape}')

## Benchmark Inference Speed

In [ ]:
import time

# Benchmark TFLite
iterations = 100
times = []

for _ in range(iterations):
    start = time.time()
    interpreter.set_tensor(input_details[0]['index'], test_input)
    interpreter.invoke()
    _ = interpreter.get_tensor(output_details[0]['index'])
    times.append(time.time() - start)

avg_time = np.mean(times) * 1000
print(f'TFLite average inference time: {avg_time:.2f}ms')
print(f'FPS: {1000/avg_time:.1f}')

# Benchmark original model
times_original = []
for _ in range(iterations):
    start = time.time()
    _ = model_mobile.predict(test_input, verbose=0)
    times_original.append(time.time() - start)

avg_time_original = np.mean(times_original) * 1000
print(f'\nOriginal model average time: {avg_time_original:.2f}ms')
print(f'Speedup: {avg_time_original/avg_time:.2f}x')

## Summary

In [ ]:
print('='*60)
print('TFLITE CONVERSION SUMMARY')
print('='*60)
print(f'\nMobileNetV2:')
print(f'  Size: {mobile_size:.2f} MB')
print(f'  Inference: {avg_time:.2f}ms')
print(f'  Ready for Raspberry Pi ✅')
print(f'\nResNet50V2:')
print(f'  Size: {resnet_size:.2f} MB')
print(f'  Best accuracy (98.74%)')
print(f'  For cloud/server deployment ✅')
print('\nFiles saved to models/ directory')
print('='*60)